In [35]:
import pandas as pd
from datasets import load_from_disk

test_dataset = load_from_disk('/opt/ml/data/test_dataset')
test_dataset = test_dataset['validation']

test_data = []

for i, row in enumerate(test_dataset):
    test_data.append(row)

test_df = pd.DataFrame(test_data)
test_df['test_id_no'] = test_df['id'].apply(lambda x: x.split('-')[-1])

def get_klue_df(df, data_source):
    data_objs = {
        'version': [],
        'title': [],
        'context': [],
        'question': [],
        'answer_text': [],
        'answer_start': [],
        'question_type': [],
        'is_impossible': [],
        'guid': [],
        'klue_id_no': [],
        'news_category': [],
        'source': [],
        'data_source': []
    }

    for i, row in df.iterrows():
        data = row['data']
        qas = data['paragraphs'][0]['qas']
        for q in qas:
            if len(q['answers']) == 0:
                continue
            data_objs['version'].append(row['version'])
            data_objs['title'].append(data['title'])
            data_objs['context'].append(data['paragraphs'][0]['context'])
            data_objs['question'].append(q['question'])
            answer_text_list = []
            answer_start_list = []
            for ans in q['answers']:
                answer_text_list.append(ans['text'])
                answer_start_list.append(ans['answer_start'])
            # data_objs['answer_text'].append(q['answers'][0]['text'])
            # data_objs['answer_start'].append(q['answers'][0]['answer_start'])
            data_objs['answer_text'].append(answer_text_list)
            data_objs['answer_start'].append(answer_start_list)        
            data_objs['question_type'].append(q['question_type'])
            data_objs['is_impossible'].append(q['is_impossible'])
            data_objs['guid'].append(q['guid'])
            data_objs['klue_id_no'].append(q['guid'].split('_')[-1])
            data_objs['news_category'].append(data['news_category'])
            data_objs['source'].append(data['source'])
            data_objs['data_source'].append(data_source)

    return pd.DataFrame(data_objs)

klue_train_df = get_klue_df(pd.read_json('./klue-mrc-v1.1_train.json'), 'train')
klue_dev_df = get_klue_df(pd.read_json('./klue-mrc-v1.1_dev.json'), 'dev')

klue_df = pd.concat([klue_train_df, klue_dev_df])

merge_df = pd.merge(test_df, klue_df, how='outer', on='question')
merge_df['id'].fillna('miss', inplace=True)
merge_df['answer_text'].fillna('none', inplace=True)
merge_df = merge_df.loc[merge_df['id'] != 'miss', :]
merge_df = merge_df.loc[merge_df['answer_text'] != 'none', :]
merge_df['answer_text'] = merge_df['answer_text'].apply(lambda x: x[0])
merge_df['answer_start'] = merge_df['answer_start'].apply(lambda x: x[0])
merge_df.to_csv('test_merge_data.csv', index=False)

In [36]:
import pandas as pd
from datasets import Dataset

test_df = pd.read_csv('./test_merge_data.csv')
test_df = test_df[['id', 'question' , 'context', 'answer_text', 'answer_start', 'title']]
test_df['document_id'] = [i for i in range(len(test_df))]
answers = []
for i, row in test_df.iterrows():
    answers.append({'answer_start': [row['answer_start']], 'text': [row['answer_text']]})
test_df['answers'] = answers
test_df.drop(columns=['answer_text', 'answer_start'], inplace=True)

test_validation_dataset = Dataset.from_pandas(test_df)
test_validation_dataset.save_to_disk('/opt/ml/data/test_validation_dataset')

,id,question,context,title,document_id,answers
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간...,더크 젠틀리의 성스러운 탐정 사무소,0,"{'answer_start': [88], 'text': ['사락사라']}"
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 ...,오퍼레이션7,1,"{'answer_start': [0], 'text': ['냉전']}"
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,"1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ...",엥겔베르트 돌푸스,2,"{'answer_start': [253], 'text': ['빌헬름 미클라스']}"
3,mrc-1-001272,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 파업을 지지했다는...,보응우옌잡,3,"{'answer_start': [771], 'text': ['프랑스']}"
4,mrc-1-000993,단공류가 일반 포유류와 다르다는 것을 알 수 있는 신체 부위는?,"단공류는 포유류의 극히 적은 부분이다. 멸종된 단공류를 제외하면, 여기에 해당하는 ...",단공류,4,"{'answer_start': [576], 'text': ['뒷다리']}"
...,...,...,...,...,...,...
470,mrc-0-000651,디즈니랜드에서 개인회원을 뽑기 시작한 년도는?,1962년부터 1965년까지의 뉴욕 세계 박람회에서 다양한 기업 창설자들과 어트랙션...,클럽 33,470,"{'answer_start': [348], 'text': ['1967년']}"
471,mrc-0-002989,타입 2 가이아 메모리을 만든 집단은?,"재단 X가 슈라우드(=소노자키 후미네)가 개발한 가이아 메모리의 테크놀로지를 분석,...",극장판 가면라이더 W FOREVER: A to Z/운명의 가이아 메모리,471,"{'answer_start': [0], 'text': ['재단 X']}"
472,mrc-0-003411,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?",콜드게임(called game)이란 야구나 소프트볼 등의 운동경기에서 심판에 의하여...,콜드게임,472,"{'answer_start': [492], 'text': ['서스펜디드 게임']}"
473,mrc-0-003436,제2캐나다기갑여단이 상륙한 곳은?,주노 해변\n\n주노 해변은 쿠르쇨르메르의 양쪽으로 뻗은 5마일 정도의 해변이었다....,제3캐나다사단,473,"{'answer_start': [7], 'text': ['주노 해변']}"


In [6]:
datasets = load_from_disk('/opt/ml/data/train_dataset')
eval_dataset = datasets['validation']
document_id = [i for i in range()]

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 240
})

In [31]:
from datasets import Dataset

test_validation_dataset = Dataset.from_pandas(test_df)
test_validation_dataset

Dataset({
    features: ['id', 'question', 'context', 'title', 'document_id', 'answers', '__index_level_0__'],
    num_rows: 475
})

In [33]:
test_validation_dataset.save_to_disk('/opt/ml/data/test_validation_dataset')

In [34]:
new_dataset = load_from_disk('/opt/ml/data/test_validation_dataset')
new_dataset

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 475
})

In [2]:
from datasets import load_from_disk

dataset = load_from_disk('/opt/ml/data/test_validation_dataset')
dataset

Dataset({
    features: ['answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 475
})